<a href="https://colab.research.google.com/github/Sabina-0219/Sentiment-Analysis-System/blob/main/Sentiment_ana.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
import urllib.request
import os
import tarfile
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
import re
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import Dense, Activation, Flatten

Using TensorFlow backend.


In [3]:
! wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
! tar zxf aclImdb_v1.tar.gz

--2022-07-02 08:45:33--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  60.2MB/s    in 1.3s    

2022-07-02 08:45:34 (60.2 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [4]:
def rm_tags(text):
  re_tag = re.compile(r'<[^>]+>')
  return re_tag.sub('', text)

In [5]:
def read_files(filetype):
  path = "/content/aclImdb/"
  file_list = []

  positive_path = path + filetype + "/pos/"
  for f in os.listdir(positive_path):
    file_list += [positive_path+f]
  
  negative_path = path + filetype + "/neg/"
  for f in os.listdir(negative_path):
    file_list += [negative_path+f]
  
  print("read", filetype, "files:", len(file_list))

  all_labels = ([1] * 12500 + [0] * 12500)

  all_texts = []
  for fi in file_list:
    with open(fi, encoding = 'utf8') as file_input:
      all_texts += [rm_tags(" ".join(file_input.readlines()))]
  return all_labels, all_texts

In [6]:
y_train, train_text = read_files("train")

read train files: 25000


In [7]:
y_test, test_text = read_files("test")

read test files: 25000


In [8]:
train_text[0]

"I saw this series when I was a kid and loved the detail it went into and never forgot it. I finally purchased the DVD collection and its just how I remembered. This is just how a doco should be, unbiased and factual. The film footage is unbelievable and the interviews are fantastic. The only other series that I have found equal to this is 'Die Deutschen Panzer'.I only wish Hollywood would sit down and watch this series, then they might make some great war movies.Note. Band of Brothers, Saving Private Ryan, Letters from Iwo Jima, Flags of Our Fathers and When Trumpets Fade are some I'd recommend"

In [9]:
y_train[0]

1

In [10]:
train_text[20000]

'This is one of the most irritating, nonsensical movies I\'ve ever had the misfortune to sit through. Every time it started to look like it might be getting good, out come more sepia tone flashbacks, followed by paranoid idiocy masquerading as social commentary. The main character, Maddox, is a manipulative, would-be rebel who lives in a mansion seemingly without any parents or responsibility. The supporting cast are all far more likeable and interesting, but are unfortunately never developed. Nor do we ever really understand the John Stanton character supposedly influencing Maddox to commit the acts of rebellion. At one point, I thought "Aha! Maddox is just nuts and is secretly making up all those communications from escaped mental patient Stanton! Now we\'re getting somewhere!" but of course, that ends up to not be the case and the whole movie turns out to be pointless, both from Maddox\'s perspective and the viewer\'s. Where\'s Ferris Bueller when we need him?'

In [12]:
y_train[20000]

0

In [13]:
token = Tokenizer(num_words = 2000)
token.fit_on_texts(train_text)

In [14]:
print(token.document_count)

25000


In [16]:
x_train_seq = token.texts_to_sequences(train_text)
x_test_seq = token.texts_to_sequences(test_text)

In [17]:
print(train_text[0])

I saw this series when I was a kid and loved the detail it went into and never forgot it. I finally purchased the DVD collection and its just how I remembered. This is just how a doco should be, unbiased and factual. The film footage is unbelievable and the interviews are fantastic. The only other series that I have found equal to this is 'Die Deutschen Panzer'.I only wish Hollywood would sit down and watch this series, then they might make some great war movies.Note. Band of Brothers, Saving Private Ryan, Letters from Iwo Jima, Flags of Our Fathers and When Trumpets Fade are some I'd recommend


In [18]:
print(x_train_seq[0])

[9, 215, 10, 197, 50, 9, 12, 3, 549, 2, 442, 1, 1584, 8, 430, 79, 2, 111, 8, 9, 413, 1, 284, 1587, 2, 90, 39, 85, 9, 10, 6, 39, 85, 3, 140, 25, 2, 1, 18, 923, 6, 1295, 2, 1, 22, 772, 1, 60, 81, 197, 11, 9, 24, 254, 5, 10, 6, 9, 60, 653, 358, 58, 864, 176, 2, 102, 10, 197, 91, 32, 234, 93, 45, 83, 321, 98, 849, 1135, 4, 1087, 1896, 1947, 35, 4, 259, 2, 50, 22, 45, 470, 382]


In [19]:
x_train = sequence.pad_sequences(x_train_seq, maxlen = 200)
x_test = sequence.pad_sequences(x_test_seq, maxlen = 200)

In [20]:
print('length before processing = ', len(x_train_seq[5]))
print(x_train_seq[5])

length before processing =  95
[203, 206, 73, 3, 190, 333, 122, 233, 9, 215, 37, 197, 3, 167, 149, 591, 10, 18, 431, 283, 37, 7, 37, 114, 633, 242, 1, 18, 12, 34, 1660, 173, 2, 252, 171, 146, 2, 170, 55, 113, 1134, 407, 39, 5, 93, 10, 18, 27, 4, 1, 114, 203, 122, 106, 1458, 62, 42, 123, 528, 257, 665, 1787, 2, 32, 62, 89, 1, 18, 212, 278, 77, 9, 442, 1, 113, 223, 117, 1974, 37, 2, 55, 750, 1438, 50, 55, 729, 5, 1, 1863, 10, 6, 18, 29, 90, 114]


In [21]:
print('length after processing = ', len(x_train[5]))
print(x_train[5])

length after processing =  200
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0  203  206   73    3  190  333  122
  233    9  215   37  197    3  167  149  591   10   18  431  283   37
    7   37  114  633  242    1   18   12   34 1660  173    2  252  171
  146    2  170   55  113 1134  407   39    5   93   10   18   27    4
    1  114  203  122  106 1458   62   42  123  528  257  665 1787    2
   32   62   89    1   18  212  278   77    9  442    1  113  223  117
 1974   37    2   55  750 1438   50   55  729 

In [22]:
model = Sequential()

In [23]:
model.add(Embedding(output_dim = 32,
                    input_dim = 2000,
                    input_length = 200))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [24]:
model.add(Flatten())

In [25]:
model.add(Dense(units = 256, 
                activation = 'relu'))

In [26]:
model.add(Dense(units = 1, 
                activation = 'sigmoid'))

In [27]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 200, 32)           64000     
_________________________________________________________________
flatten_1 (Flatten)          (None, 6400)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               1638656   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 1,702,913
Trainable params: 1,702,913
Non-trainable params: 0
_________________________________________________________________


In [28]:
model.compile(loss = 'binary_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [29]:
train_history = model.fit(x_train, y_train, batch_size = 100, epochs = 10, verbose = 0, validation_split = 0.2)

In [30]:
scores = model.evaluate(x_test, y_test, verbose=1)

25000/25000 [==============================] - 1s 45us/step


In [31]:
scores[1]

0.8363199830055237